In [24]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [25]:
# !pip install -U seaborn  # if you don't have it, or pip3 for python3

In [28]:
print(min(data['DateOfDeparture']))
print(max(data['DateOfDeparture']))

2011-09-01
2013-03-05


In [29]:
data.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,log_PAX,std_wtd
0,2012-06-19,ORD,DFW,12.875000,12.331296,9.812647
1,2012-09-10,LAS,DEN,14.285714,10.775182,9.466734
2,2012-10-05,DEN,LAX,10.863636,11.083177,9.035883
3,2011-10-09,ATL,ORD,11.480000,11.169268,7.990202
4,2012-02-21,DEN,SFO,11.450000,11.269364,9.517159


In [30]:
data['Departure'].unique()

array(['ORD', 'LAS', 'DEN', 'ATL', 'SFO', 'EWR', 'IAH', 'LAX', 'DFW',
       'SEA', 'JFK', 'PHL', 'MIA', 'DTW', 'BOS', 'MSP', 'CLT', 'MCO',
       'PHX', 'LGA'], dtype=object)

In [ ]:
data_encoded_sorted = data_encoded.sort_values(by='month')
sns.boxplot(x = 'month',
            y = 'log_PAX',
            data = data_encoded_sorted,
            color = 'Red');

In [ ]:
data.describe()

In [ ]:
data.dtypes

In [12]:
data.shape

(8902, 6)

In [13]:
print(data['log_PAX'].mean())
print(data['log_PAX'].std())

10.999047672120955
0.9939452411746512


## Preprocessing for prediction

Getting dates into numerical columns is a common operation when time series are analyzed with non-parametric predictors. The code below makes all possible choices: ordered columns for the year, month, day, weekday, week, and day in the year, and one-hot columns for year month, day, weekday, and week.

The departure and arrival airports are also converted into one-hot columns. 

In [14]:
data_encoded = data

data_encoded = data_encoded.join(pd.get_dummies(data_encoded['Departure'], prefix='d'))
data_encoded = data_encoded.join(pd.get_dummies(data_encoded['Arrival'], prefix='a'))
#data_encoded = data_encoded.drop('Departure', axis=1)
#data_encoded = data_encoded.drop('Arrival', axis=1)

# following http://stackoverflow.com/questions/16453644/regression-with-date-variable-using-scikit-learn
data_encoded['DateOfDeparture'] = pd.to_datetime(data_encoded['DateOfDeparture'])
data_encoded['year'] = data_encoded['DateOfDeparture'].dt.year
data_encoded['month'] = data_encoded['DateOfDeparture'].dt.month
data_encoded['day'] = data_encoded['DateOfDeparture'].dt.day
data_encoded['weekday'] = data_encoded['DateOfDeparture'].dt.weekday
data_encoded['week'] = data_encoded['DateOfDeparture'].dt.week
data_encoded['n_days'] = data_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

data_encoded = data_encoded.join(pd.get_dummies(data_encoded['year'], prefix='y'))
data_encoded = data_encoded.join(pd.get_dummies(data_encoded['month'], prefix='m'))
data_encoded = data_encoded.join(pd.get_dummies(data_encoded['day'], prefix='d'))
data_encoded = data_encoded.join(pd.get_dummies(data_encoded['weekday'], prefix='wd'))
data_encoded = data_encoded.join(pd.get_dummies(data_encoded['week'], prefix='w'))

### Import et travaille de la base Externe

In [57]:
#Add New Features GH
import pandas as pd
import os
X_encoded = pd.read_csv("submissions/starting_kit/external_data.csv")
#data_External["Date"] = pd.to_datetime(data_External.Date, format = '%Y-%m-%d')
X_encoded.head(5)

,Unnamed: 0,Date,Departure,DepMaxTemperatureC,DepMeanTemperatureC,DepMinTemperatureC,DepDewPointC,DepMeanDewPointC,DepMinDewpointC,DepMaxHumidity,DepMeanHumidity,DepMinHumidity,DepMaxSeaLevelPressurehPa,DepMeanSeaLevelPressurehPa,DepMinSeaLevelPressurehPa,DepMaxVisibilityKm,DepMeanVisibilityKm,DepMinVisibilitykM,DepMaxWindSpeedKmH,DepMeanWindSpeedKmH,DepMaxGustSpeedKmH,DepPrecipitationmm,DepCloudCover,DepEvents,DepWindDirDegrees,Arrival,ArrMaxTemperatureC,ArrMeanTemperatureC,ArrMinTemperatureC,ArrDewPointC,ArrMeanDewPointC,ArrMinDewpointC,ArrMaxHumidity,ArrMeanHumidity,ArrMinHumidity,ArrMaxSeaLevelPressurehPa,ArrMeanSeaLevelPressurehPa,ArrMinSeaLevelPressurehPa,ArrMaxVisibilityKm,ArrMeanVisibilityKm,ArrMinVisibilitykM,ArrMaxWindSpeedKmH,ArrMeanWindSpeedKmH,ArrMaxGustSpeedKmH,ArrPrecipitationmm,ArrCloudCover,ArrEvents,ArrWindDirDegrees,Holiday,DepLatitude,DepLongitude,DepState,DepPop_2010,DepPop_2016,DepPop_2015,DepAge_median,DepCompanies,DepGraduates,DepHousings,DepIncome,DepForeigners,DepPoverty,DepVeterans,ArrLatitude,ArrLongitude,ArrState,ArrPop_2010,ArrPop_2016,ArrPop_2015,ArrAge_median,ArrCompanies,ArrGraduates,ArrHousings,ArrIncome,ArrForeigners,ArrPoverty,ArrVeterans
0,0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,0,33.641,-84.4226,Georgia,420003,472522,448901,33.4,64593,89.0,228579,47527,32701,24.6,20009,33.641,-84.4226,Georgia,420003,472522,448901,33.4,64593,89.0,228579,47527,32701,24.6,20009
1,1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,0,33.641,-84.4226,Georgia,420003,472522,448901,33.4,64593,89.0,228579,47527,32701,24.6,20009,33.641,-84.4226,Georgia,420003,472522,448901,33.4,64593,89.0,228579,47527,32701,24.6,20009
2,2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,33.641,-84.4226,Georgia,420003,472522,448901,33.4,64593,89.0,228579,47527,32701,24.6,20009,33.641,-84.4226,Georgia,420003,472522,448901,33.4,64593,89.0,228579,47527,32701,24.6,20009
3,3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,33.641,-84.4226,Georgia,420003,472522,448901,33.4,64593,89.0,228579,47527,32701,24.6,20009,33.641,-84.4226,Georgia,420003,472522,448901,33.4,64593,89.0,228579,47527,32701,24.6,20009
4,4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,33.641,-84.4226,Georgia,420003,472522,448901,33.4,64593,89.0,228579,47527,32701,24.6,20009,33.641,-84.4226,Georgia,420003,472522,448901,33.4,64593,89.0,228579,47527,32701,24.6,20009


In [59]:
X_encoded.shape

(220800, 77)

### Merge des données Externes

#### Merge Données de temperatures

In [93]:
data = pd.read_csv("data/train.csv.bz2")
y_array = data['log_PAX'].values
X_df = data.drop('log_PAX', axis=1)

In [94]:
y_array.shape


(8902,)

In [96]:
import pandas as pd
import numpy as np
import os

# from pandas.tseries.holiday import Holiday, USMemorialDay, AbstractHolidayCalendar, nearest_workday, MO
from copy import deepcopy
from datetime import datetime, timedelta

# Define a mapped version of the Haversine formula to compute distances between airports. 
# Inspired from https://stackoverflow.com
#    /questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
from math import radians, cos, sin, asin, sqrt

def haversine(row):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # we map the rows
    lon1 = row['DepLongitude']
    lat1 = row['DepLatitude']
    lon2 = row['ArrLongitude']
    lat2 = row['ArrLatitude']
    
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

##

print(X_encoded.head())
X_encoded = X_df
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))

# following http://stackoverflow.com/questions/16453644/regression-with-date-variable-using-scikit-learn
X_encoded['Date'] = X_encoded['DateOfDeparture']
X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
X_encoded['year'] = X_encoded['DateOfDeparture'].dt.year
X_encoded['month'] = X_encoded['DateOfDeparture'].dt.month
X_encoded['day'] = X_encoded['DateOfDeparture'].dt.day
X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
X_encoded['week'] = X_encoded['DateOfDeparture'].dt.week
X_encoded['n_days'] = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)


X_encoded = X_encoded.join(pd.get_dummies(X_encoded['year'], prefix='y'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['month'], prefix='m'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['day'], prefix='d'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['week'], prefix='w'))

print(X_encoded.head())

if True:
    ## print(path)
    external_data = pd.read_csv("submissions/starting_kit/external_data.csv")
    X_encoded = pd.merge(
        X_encoded, external_data, how='left',
        left_on=['Date', 'Departure', 'Arrival'],
        right_on=['Date', 'Departure', 'Arrival'],
        sort=False)


X_encoded = X_encoded.drop('Arrival', axis=1)
X_encoded = X_encoded.drop('DateOfDeparture', axis=1)

# X_encoded = X_encoded.drop('WeeksToDeparture', axis=1)
# X_encoded = X_encoded.drop('std_wtd', axis=1)

if True:
    X_encoded = X_encoded.drop('Date', axis=1)
    X_encoded = X_encoded.drop('DepEvents', axis=1)
    X_encoded = X_encoded.drop('ArrEvents', axis=1)
    X_encoded = X_encoded.drop('DepMaxGustSpeedKmH', axis=1)
    X_encoded = X_encoded.drop('ArrMaxGustSpeedKmH', axis=1)
    X_encoded = X_encoded.drop('DepPrecipitationmm', axis=1)
    X_encoded = X_encoded.drop('ArrPrecipitationmm', axis=1)
    X_encoded = X_encoded.drop('DepState', axis=1)
    X_encoded = X_encoded.drop('ArrState', axis=1)


           
X_array = X_encoded.values



   Unnamed: 0        Date Departure  DepMaxTemperatureC  DepMeanTemperatureC  \
0           0  2011-09-01       ATL                  35                   29   
1           1  2011-09-02       ATL                  36                   29   
2           2  2011-09-03       ATL                  35                   29   
3           3  2011-09-04       ATL                  27                   24   
4           4  2011-09-05       ATL                  26                   24   

   DepMinTemperatureC  DepDewPointC  DepMeanDewPointC  DepMinDewpointC  \
0                  24            21                18               14   
1                  22            17                15               14   
2                  23            17                16               14   
3                  22            22                19               16   
4                  22            23                22               20   

   DepMaxHumidity  DepMeanHumidity  DepMinHumidity  DepMaxSeaLevelPressure

  DateOfDeparture Departure Arrival  WeeksToDeparture   std_wtd  d_ATL  d_BOS  \
0      2012-06-19       ORD     DFW         12.875000  9.812647      0      0   
1      2012-09-10       LAS     DEN         14.285714  9.466734      0      0   
2      2012-10-05       DEN     LAX         10.863636  9.035883      0      0   
3      2011-10-09       ATL     ORD         11.480000  7.990202      1      0   
4      2012-02-21       DEN     SFO         11.450000  9.517159      0      0   

   d_CLT  d_DEN  d_DFW  d_DTW  d_EWR  d_IAH  d_JFK  d_LAS  d_LAX  d_LGA  \
0      0      0      0      0      0      0      0      0      0      0   
1      0      0      0      0      0      0      0      1      0      0   
2      0      1      0      0      0      0      0      0      0      0   
3      0      0      0      0      0      0      0      0      0      0   
4      0      1      0      0      0      0      0      0      0      0   

   d_MCO  d_MIA  d_MSP  d_ORD  d_PHL  d_PHX  d_SEA  d_SFO  a_A

In [139]:
import pandas as pd
import numpy as np
import os

# from pandas.tseries.holiday import Holiday, USMemorialDay, AbstractHolidayCalendar, nearest_workday, MO
from copy import deepcopy
from datetime import datetime, timedelta

# Define a mapped version of the Haversine formula to compute distances between airports. 
# Inspired from https://stackoverflow.com
#    /questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
from math import radians, cos, sin, asin, sqrt

def haversine(row):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # we map the rows
    lon1 = row['DepLongitude']
    lat1 = row['DepLatitude']
    lon2 = row['ArrLongitude']
    lat2 = row['ArrLatitude']
    
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

##

## print(X_encoded.head())
X_encoded = X_df
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))

# following http://stackoverflow.com/questions/16453644/regression-with-date-variable-using-scikit-learn
X_encoded['Date'] = X_encoded['DateOfDeparture']
X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
X_encoded['year'] = X_encoded['DateOfDeparture'].dt.year
X_encoded['month'] = X_encoded['DateOfDeparture'].dt.month
X_encoded['day'] = X_encoded['DateOfDeparture'].dt.day
X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
X_encoded['week'] = X_encoded['DateOfDeparture'].dt.week
X_encoded['n_days'] = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)


X_encoded = X_encoded.join(pd.get_dummies(X_encoded['year'], prefix='y'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['month'], prefix='m'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['day'], prefix='d'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['week'], prefix='w'))

## print(X_encoded.head())

if True:
    ## print(path)
    external_data = pd.read_csv("submissions/starting_kit/external_data.csv")
    X_encoded = pd.merge(
        X_encoded, external_data, how='left',
        left_on=['Date', 'Departure', 'Arrival'],
        right_on=['Date', 'Departure', 'Arrival'],
        sort=False)

    ## print(X_encoded['DepEvents'].unique())

    def withFrog(Value):
        if type(Value) is str:
            if Value.find("Frog") == -1:
                return False
            else:
                return True
        else:
            return False

    def withThun(Value):
        if type(Value) is str:
            if Value.find("Thunderstorm") == -1:
                return False
            else:
                return True
        else:
            return False

    def withSnow(Value):
        if type(Value) is str:
            if Value.find("Snow") == -1:
                return False
            else:
                return True
        else:
            return False

    def withRain(Value):
        if type(Value) is str:
            if Value.find("Rain") == -1:
                return False
            else:
                return True
        else:
            return False


    X_encoded['DepFrog'] = X_encoded['DepEvents'].apply(withFrog)
    X_encoded['DepThun'] = X_encoded['DepEvents'].apply(withThun)
    X_encoded['DepSnow'] = X_encoded['DepEvents'].apply(withSnow)
    X_encoded['DepRain'] = X_encoded['DepEvents'].apply(withRain)
    X_encoded['ArrFrog'] = X_encoded['ArrEvents'].apply(withFrog)
    X_encoded['ArrThun'] = X_encoded['ArrEvents'].apply(withThun)
    X_encoded['ArrSnow'] = X_encoded['ArrEvents'].apply(withSnow)
    X_encoded['ArrRain'] = X_encoded['ArrEvents'].apply(withRain)

X_encoded.to_csv("X_array_2.csv")

## Suppression des colonnes non numériques

X_encoded = X_encoded.drop('Departure', axis=1)
X_encoded = X_encoded.drop('Arrival', axis=1)
X_encoded = X_encoded.drop('DateOfDeparture', axis=1)

# X_encoded = X_encoded.drop('WeeksToDeparture', axis=1)
# X_encoded = X_encoded.drop('std_wtd', axis=1)

if True:
    X_encoded = X_encoded.drop('Date', axis=1)
    X_encoded = X_encoded.drop('DepEvents', axis=1)
    X_encoded = X_encoded.drop('ArrEvents', axis=1)
    X_encoded = X_encoded.drop('DepMaxGustSpeedKmH', axis=1)
    X_encoded = X_encoded.drop('ArrMaxGustSpeedKmH', axis=1)
    X_encoded = X_encoded.drop('DepPrecipitationmm', axis=1)
    X_encoded = X_encoded.drop('ArrPrecipitationmm', axis=1)
    X_encoded = X_encoded.drop('DepState', axis=1)
    X_encoded = X_encoded.drop('ArrState', axis=1)


X_encoded['Distance'] = X_encoded.apply(lambda row: haversine(row), axis=1)

if True:
    m = X_encoded.isnull().any()
    # print("========= COLUMNS WITH NULL VALUES =================")
    # print(m[m])
    m = np.isfinite(X_encoded.select_dtypes(include=['float64'])).any()
    # print("========= COLUMNS WITH INFINITE VALUES =================")
    # print(m[m])
        
X_array = X_encoded.values



In [138]:
X_encoded.head(5)

,WeeksToDeparture,std_wtd,d_ATL,d_BOS,d_CLT,d_DEN,d_DFW,d_DTW,d_EWR,d_IAH,d_JFK,d_LAS,d_LAX,d_LGA,d_MCO,d_MIA,d_MSP,d_ORD,d_PHL,d_PHX,d_SEA,d_SFO,a_ATL,a_BOS,a_CLT,a_DEN,a_DFW,a_DTW,a_EWR,a_IAH,a_JFK,a_LAS,a_LAX,a_LGA,a_MCO,a_MIA,a_MSP,a_ORD,a_PHL,a_PHX,a_SEA,a_SFO,year,month,day,weekday,week,n_days,y_2011,y_2012,y_2013,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,m_11,m_12,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,wd_0,wd_1,wd_2,wd_3,wd_4,wd_5,wd_6,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17,w_18,w_19,w_20,w_21,w_22,w_23,w_24,w_25,w_26,w_27,w_28,w_29,w_30,w_31,w_32,w_33,w_34,w_35,w_36,w_37,w_38,w_39,w_40,w_41,w_42,w_43,w_44,w_45,w_46,w_47,w_48,w_49,w_50,w_51,w_52,Unnamed: 0,DepMaxTemperatureC,DepMeanTemperatureC,DepMinTemperatureC,DepDewPointC,DepMeanDewPointC,DepMinDewpointC,DepMaxHumidity,DepMeanHumidity,DepMinHumidity,DepMaxSeaLevelPressurehPa,DepMeanSeaLevelPressurehPa,DepMinSeaLevelPressurehPa,DepMaxVisibilityKm,DepMeanVisibilityKm,DepMinVisibilitykM,DepMaxWindSpeedKmH,DepMeanWindSpeedKmH,DepCloudCover,DepWindDirDegrees,ArrMaxTemperatureC,ArrMeanTemperatureC,ArrMinTemperatureC,ArrDewPointC,ArrMeanDewPointC,ArrMinDewpointC,ArrMaxHumidity,ArrMeanHumidity,ArrMinHumidity,ArrMaxSeaLevelPressurehPa,ArrMeanSeaLevelPressurehPa,ArrMinSeaLevelPressurehPa,ArrMaxVisibilityKm,ArrMeanVisibilityKm,ArrMinVisibilitykM,ArrMaxWindSpeedKmH,ArrMeanWindSpeedKmH,ArrCloudCover,ArrWindDirDegrees,Holiday,DepLatitude,DepLongitude,DepPop_2010,DepPop_2016,DepPop_2015,DepAge_median,DepCompanies,DepGraduates,DepHousings,DepIncome,DepForeigners,DepPoverty,DepVeterans,ArrLatitude,ArrLongitude,ArrPop_2010,ArrPop_2016,ArrPop_2015,ArrAge_median,ArrCompanies,ArrGraduates,ArrHousings,ArrIncome,ArrForeigners,ArrPoverty,ArrVeterans,DepFrog,DepThun,DepSnow,DepRain,ArrFrog,ArrThun,ArrSnow,ArrRain,Distance
0,12.875000,9.812647,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2012,6,19,1,25,15510,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33964,35,31,26,20,18,17,62,51,39,1014,1012,1009,16,16,16,37,25,1,208,34,29,24,22,21,19,82,63,44,1012,1010,1009,16,16,16,48,29,5,161,0,41.9796,-87.9045,2695598,2704958,2717534,33.7,291007,82.3,1192544,48522,572066,22.3,8284,32.8959,-97.0372,1197816,1317929,1260688,32.4,142658,74.5,533556,43781,305921,24.0,467,False,False,False,False,False,False,False,False,1290.782275
1,14.285714,9.466734,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2012,9,10,0,37,15593,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,48951,36,32,27,16,13,10,51,36,21,1011,1009,1005,16,15,11,32,18,5,187,33,25,16,-2,-6,-8,21,14,7,1011,1008,1005,16,16,16,35,15,3,207,0,36.0852,-115.1507,583756,632912,605097,36.9,55856,83.3,250279,50202,127458,17.5,45145,39.8589,-104.6733,600158,69306,649654,34.1,79097,86.1,294191,53637,10437,17.3,32249,False,False,False,False,False,False,False,False,1008.860199
2,10.863636,9.035883,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2012,10,5,4,40,15618,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,24688,6,2,-1,0,-2,-4,92,76,60,1027,1023,1020,16,9,1,27,15,7,357,22,19,16,17,16,14,93,77,61,1018,1016,1014,16,13,8,24,8,5,266,0,39.8589,-104.6733,600158,69306,649654,34.1,79097,86.1,294191,53637,10437,17.3,32249,33.9425,-118.4090,3792621,3976322,3900794,34.9,497999,75.5,1436543,50205,1489926,22.1,97236,False,False,True,True,False,False,F

array([13.590909090899999, 11.375240827999999, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 2012, 1, 10, 1, 2, 15349, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 66371, 17, 14, 12, 14, 13, 11, 100, 86, 72,
       1019, 1016, 1011, 16, 8, 0, 23, 8, 7, 183, 15, 10, 4, 6, 4, 2, 89,
       67, 44, 1022, 1021, 1019, 16, 13, 10, 21, 5, 3, 244, 0, 33.641,
       -84.4226, 420003, 472522, 448901, 33.4, 64593, 89.0, 228579, 47527,
       32701, 24.6, 20009, 37.6218, -122.37899999999999, 805235, 870887,
       840763, 38.5, 116803, 87.0, 383676, 81294, 295417, 13.2, 26401,
       False, False, False, True, F

In [99]:
X_encoded.shape

(8902, 228)

### A linear regressor baseline

We drop the target column and the original data column.

In [101]:
#features = X_encoded.drop(['log_PAX','DateOfDeparture','Arrival','Departure'], axis=1)
#X_columns = X_encoded.columns.drop(['log_PAX','DateOfDeparture','Arrival','Departure'])
X = X_array
y = y_array
print(X)

[[12.875 9.81264711881 0 ..., False False 1290.7822750849482]
 [14.2857142857 9.466733735510001 0 ..., False False 1008.860198886347]
 [10.8636363636 9.03588325219 0 ..., False False 1385.0669964547446]
 ..., 
 [11.0476190476 7.90870526999 0 ..., False False 665.2492317063751]
 [6.076923076919999 4.03033370127 0 ..., True False 1088.1860347830186]
 [9.52631578947 6.167733430730001 0 ..., False False 958.7848373263851]]


In [102]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

In [103]:
X_train.item

<function ndarray.item>

It gives us a pretty nice improvement above baseline

In [104]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

reg = LinearRegression()

scores = cross_val_score(reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print("log RMSE: {:.4f} +/-{:.4f}".format(
    np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

log RMSE: 0.6005 +/-0.0066


Exercise: Visualize the coefficients, try to make sense of them.

## Random Forests

In [105]:
%%time
from sklearn.ensemble import RandomForestRegressor


Wall time: 125 ms


In [106]:
%%time
n_estimators = 500
max_depth = 100  #la profondeur ameliore le score mais je risque d'overfitter non ?
min_samples_split=6 # nombre d'exemple dans la feuille oùtu arrète de couper
max_features = 15
reg = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features)

scores = cross_val_score(reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error',n_jobs=3)
print("log RMSE: {:.4f} +/-{:.4f}".format(
    np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

log RMSE: 0.5184 +/-0.0139


## XGBOOST Test

In [107]:
import xgboost as xgb
from xgboost import XGBRegressor

In [120]:
%%time
reg = XGBRegressor(max_depth=15, learning_rate=0.02, n_estimators=500,subsample=0.8, colsample_bytree=0.8, min_child_weight=5)

scores = cross_val_score(reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error',n_jobs=3)
print("log RMSE: {:.4f} +/-{:.4f}".format(
    np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

log RMSE: 0.3718 +/-0.0068
Wall time: 2min 10s


In [121]:
X_train.shape

(7121, 228)

In [125]:
reg.fit(X_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.02, max_delta_step=0, max_depth=15,
       min_child_weight=5, missing=None, n_estimators=500, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8)

In [133]:
reg.booster().get_score(importance_type = 'gain')

#plot_importance()

{'f0': 1.026318228344446,
 'f1': 0.17784838193422173,
 'f10': 2.048692004045801,
 'f100': 0.899114186595745,
 'f101': 1.894560009677419,
 'f102': 0.20610616419753086,
 'f103': 0.12752285714285713,
 'f104': 0.3570360393939394,
 'f105': 0.313145465652174,
 'f106': 0.36145678148148147,
 'f107': 0.2737299444444443,
 'f108': 0.23195430999999994,
 'f109': 0.2058321996039604,
 'f11': 0.5407442798857143,
 'f110': 0.7589844291666669,
 'f111': 0.15387725937500002,
 'f112': 0.06330830600000001,
 'f113': 0.18107096153846153,
 'f114': 0.8163367485915494,
 'f115': 0.19794714374999997,
 'f116': 0.07576373846153846,
 'f117': 0.10268216315789473,
 'f118': 0.03212453333333334,
 'f119': 0.10285584444444446,
 'f12': 0.840589738947368,
 'f120': 0.15585007301587298,
 'f121': 0.7137425630434779,
 'f122': 0.7953446506493506,
 'f123': 0.07615246153846154,
 'f124': 0.0697036,
 'f125': 0.11257612999999997,
 'f126': 0.1585029166666667,
 'f127': 2.5567603414666666,
 'f128': 0.21272004166666666,
 'f129': 0.06414871

In [124]:
plt.figure(figsize=(15, 5))

ordering = np.argsort(reg.feature_importances_)[::-1][:50]

importances = reg.feature_importances_[ordering]
feature_names = X_columns[ordering]

x = np.arange(len(feature_names))
plt.bar(x, importances)
plt.xticks(x + 0.5, feature_names, rotation=90, fontsize=15);

AttributeError: 'XGBRegressor' object has no attribute 'feature_importances_'

### Random Search

In [110]:
from sklearn.model_selection import RandomizedSearchCV

In [111]:
param_distributions = {'learning_rate' : [0.1,0.05,0.02],
                       'max_depth': [3,6,9,18],
                       'n_estimators': [100,200,500],
                       'subsample': [0.5,0.8,1],
                       'colsample_bytree' : [0.5,0.8,1], 
                       'min_child_weight': [2,5,10,50]}

In [112]:
search = RandomizedSearchCV(reg,param_distributions,n_iter=1, scoring='neg_mean_squared_error', cv=3)

In [113]:
%%time
search.fit(X_train,y_train)

Wall time: 2min 1s


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.02, max_delta_step=0, max_depth=18,
       min_child_weight=5, missing=None, n_estimators=500, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8),
          fit_params=None, iid=True, n_iter=1, n_jobs=1,
          param_distributions={'learning_rate': [0.1, 0.05, 0.02], 'max_depth': [3, 6, 9, 18], 'n_estimators': [100, 200, 500], 'subsample': [0.5, 0.8, 1], 'colsample_bytree': [0.5, 0.8, 1], 'min_child_weight': [2, 5, 10, 50]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring='neg_mean_squared_error',
          verbose=0)

In [114]:
search.best_params_

{'colsample_bytree': 0.8,
 'learning_rate': 0.05,
 'max_depth': 18,
 'min_child_weight': 5,
 'n_estimators': 500,
 'subsample': 0.8}

In [115]:
search.grid_scores_

C:\Users\Guillaume\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: -0.14838, std: 0.00356, params: {'subsample': 0.8, 'n_estimators': 500, 'min_child_weight': 5, 'max_depth': 18, 'learning_rate': 0.05, 'colsample_bytree': 0.8}]

In [116]:
print("log RMSE: {:.4f} +/-{:.4f}".format(
    np.mean(np.sqrt(-search.best_score_)), np.std(np.sqrt(-search.best_score_))))

log RMSE: 0.3852 +/-0.0000


## Variable importances

In [113]:
reg.fit(X_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.02, max_delta_step=0, max_depth=18,
       min_child_weight=5, missing=None, n_estimators=500, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8)

In [116]:
len(X_columns)

156

## Building predictive models

### The feature extractor

The feature extractor implements a single <code>transform</code> function. It receives the full pandas object X_df (without the labels). It should produce a numpy array representing the features extracted. If you want to use the (training) labels to save some state of the feature extractor, you can do it in the fit function.

The starting kit feature extractor shows you how to join your data to external data. You will have the possibility to submit a single external csv for each of your submission (so if you have several data sets, you first have to do the join offline, and save it as a csv). In this case it is whether data, joined to the database on the <code>DateOfDeparture</code> and <code>Arrival</code> fields. Attention: when you join the data, make sure that the <b><font color=red>order</font> of the rows in the data frame does not change</b>.

The following cell is *not* executed in the notebook, rather it saves its content in the file in the first line, so you can edit your submission before running the local test below and submitting it at the RAMP site.

In [166]:
#GH
#Externaldata = pd.read_csv("submissions/starting_kit/external_data.csv")
#Externaldata.head()

In [1]:
%%file submissions/starting_kit/feature_extractor.py
import pandas as pd
import os


class FeatureExtractor(object):
    def __init__(self):
        pass

    def fit(self, X_df, y_array):
        pass

    def transform(self, X_df):
        X_encoded = X_df
        #Transfo X_df
        X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
        X_encoded['year'] = X_encoded['DateOfDeparture'].dt.year
        X_encoded['month'] = X_encoded['DateOfDeparture'].dt.month
        X_encoded['day'] = X_encoded['DateOfDeparture'].dt.day
        X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
        X_encoded['week'] = X_encoded['DateOfDeparture'].dt.week
        X_encoded['n_days'] = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['year'], prefix='y'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['month'], prefix='m'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['day'], prefix='d'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['week'], prefix='w'))
        
        #External
        path = os.path.dirname(__file__)
        data_External = pd.read_csv(os.path.join(path, 'external_data.csv'))
        data_External["DateOfDeparture"] = pd.to_datetime(data_External.DateOfDeparture, format = '%Y-%m-%d')
        data_External = data_External.drop('log_PAX', axis=1)
        data_External = data_External.drop('std_wtd', axis=1)
        data_External = data_External.drop('WeeksToDeparture', axis=1)
        data_External = data_External.drop('Unnamed: 0', axis=1)
        
        X_encoded = pd.merge(X_encoded,
                             data_External, 
                             how='left',
                             left_on=['DateOfDeparture', 'Departure','Arrival'],
                             right_on=['DateOfDeparture', 'Departure','Arrival'],
                             sort = False)
        
        X_encoded = X_encoded.drop('Departure', axis=1)
        X_encoded = X_encoded.drop('Arrival', axis=1)
        X_encoded = X_encoded.drop('DateOfDeparture', axis=1)
        
        X_array = X_encoded.values
        return X_array

Overwriting submissions/starting_kit/feature_extractor.py



### The regressor

The regressor should implement an sklearn-like regressor with fit and predict functions.

In [2]:
%%file submissions/starting_kit/regressor.py
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator
import xgboost as xgb
from xgboost import XGBRegressor


class Regressor(BaseEstimator):
    def __init__(self):
        
        self.clf = RandomForestRegressor(n_estimators=10, max_depth=10, max_features=10)
        
        
        #self.clf = reg = XGBRegressor(max_depth=18,
         #                             learning_rate=0.03,
          #                            n_estimators=50,
           #                           subsample=0.8,
            #                          colsample_bytree=1,
             #                         min_child_weight=5)
            
    def fit(self, X, y):
        self.clf.fit(X, y)

    def predict(self, X):
        return self.clf.predict(X)



Overwriting submissions/starting_kit/regressor.py


## Local testing (before submission)

It is <b><span style="color:red">important that you test your submission files before submitting them</span></b>. For this we provide a unit test. Note that the test runs on your files in [`submissions/starting_kit`](http://localhost:8890/tree/submissions/starting_kit).

First `pip install ramp-workflow` or install it from the [github repo](https://github.com/paris-saclay-cds/ramp-workflow). Make sure that the python files `feature_extractor.py`, `regressor.py`, and `external_data.csv` are in the  [`submissions/starting_kit`](http://localhost:8890/tree/submissions/starting_kit) folder, and the data `train.csv.bz2` and `test.csv.bz2` are in [`data`](http://localhost:8890/tree/data). Then run

```ramp_test_submission```

If it runs and print training and test errors on each fold, then you can submit the code.

In [ ]:
!ramp_test_submission

Alternatively, load and execute `rampwf.utils.testing.py`, and call `assert_submission`. This may be useful if you would like to understand how we instantiate the workflow, the scores, the data connectors, and the cross validation scheme defined in [`problem.py`](problem.py), and how we insert and train/test your submission.

In [31]:
# %load https://raw.githubusercontent.com/paris-saclay-cds/ramp-workflow/master/rampwf/utils/testing.py

In [32]:
# assert_submission()

## Submitting to [ramp.studio](http://ramp.studio)

Once you found a good model, you can submit it to [ramp.studio](http://www.ramp.studio). First, if it is your first time using RAMP, [sign up](http://www.ramp.studio/sign_up), otherwise [log in](http://www.ramp.studio/login). Then find an open event on the particular problem, for example, the event [DSSP 6](https://www.ramp.studio/events/air_passengers_dssp6) for this RAMP. Sign up for the event. Both signups are controled by RAMP administrators, so there **can be a delay between asking for signup and being able to submit**.

Once your signup request is accepted, you can go to your [sandbox](http://www.ramp.studio/events/air_passengers_dssp6/sandbox) and copy-paste (or upload) [`feature_extractor.py`](http://localhost:8890/edit/submissions/starting_kit/feature_extractor.py), [`regressor.py`](http://localhost:8890/edit/submissions/starting_kit/regressor.py), and [`external_data.csv`](http://localhost:8890/edit/submissions/starting_kit/external_data.csv) from `submissions/starting_kit`. Save it, rename it, then submit it. The submission is trained and tested on our backend in the same way as `ramp_test_submission` does it locally. While your submission is waiting in the queue and being trained, you can find it in the "New submissions (pending training)" table in [my submissions](http://www.ramp.studio/events/air_passengers_dssp6/my_submissions). Once it is trained, you get a mail, and your submission shows up on the [public leaderboard](http://www.ramp.studio/events/air_passengers_dssp6/leaderboard). 
If there is an error (despite having tested your submission locally with `ramp_test_submission`), it will show up in the "Failed submissions" table in [my submissions](http://www.ramp.studio/events/air_passengers_dssp6/my_submissions). You can click on the error to see part of the trace.

After submission, do not forget to give credits to the previous submissions you reused or integrated into your submission.

The data set we use at the backend is usually different from what you find in the starting kit, so the score may be different.

The usual way to work with RAMP is to explore solutions, add feature transformations, select models, perhaps do some AutoML/hyperopt, etc., _locally_, and checking them with `ramp_test_submission`. The script prints mean cross-validation scores 
```
----------------------------
train rmse = 0.748 ± 0.0117
valid rmse = 0.858 ± 0.0111
test rmse = 0.881 ± 0.005
```
The official score in this RAMP (the first score column after "historical contributivity" on the [leaderboard](http://www.ramp.studio/events/air_passengers_dssp6/leaderboard)) is root mean squared error ("rmse"), so the line that is relevant in the output of `ramp_test_submission` is `valid rmse = 0.858 ± 0.0111`. When the score is good enough, you can submit it at the RAMP.

## More information

You can find more information in the [README](https://github.com/paris-saclay-cds/ramp-workflow/blob/master/README.md) of the [ramp-workflow library](https://github.com/paris-saclay-cds/ramp-workflow).

## Contact

Don't hesitate to [contact us](mailto:admin@ramp.studio?subject=air passengers notebook).